In [9]:
import os

import matplotlib.pyplot as plot
import swiftsimio

# Simulation directory structure

Before we start looking at the data let's take a quick look at how everything is stored.

All simulations are kept at `/cosma8/data/dp004/colibre/Runs`. Here you will see a collection of directories with names of the form `LXXXXNYYYY`. The value of `XXXX` corresponds to the simulation box side length in comoving Mpc, and the value of `YYYY` is the number of gas particles per linear dimension. Each simulation has four times as many dark matter particles as gas. We will use one of the `L0025N0376` simulations, meaning they have a volume of $(25\mathrm{Mpc})^3$, $376^3$ gas particles, and $4 \times 376^3$ dark matter particles.

For each box size and resolution we have a multiple simulations. Each of these has a different set of subgrid models (the way we include physical processes in the simulation, e.g. black hole feedback). For this notebook we will use the `Fiducial_test` simulation.

Let's list what's present inside the simulation directory:

In [11]:
!ls /cosma8/data/dp004/colibre/Runs/L0025N0376/Fiducial_test

colibre_L025N0376.yml	 pipeline_output	  SNII.txt
colibre.xmf		 rank_cpu_balance.log	  SOAP
data_0123.yml		 rank_hostname.log	  statistics.txt
dependency_graph_-1.csv  rank_memory_balance.log  submit.slurm
HBTplus			 resub.sh		  swift-colibre
logs			 resub.slurm		  task_level_0000_-1.txt
output_fields.yml	 r_processes.txt	  timesteps.txt
output_list_ps.txt	 SFR.txt		  unused_parameters.yml
output_list.txt		 snapshots		  used_parameters
partition_fixed_costs.h  SNIa.txt


You probably don't won't ever need to look at most of these files. The most important files/directories (which we'll look at below) are:
- `snapshots` This directory contains the particles and their properties output by the simulation at different times
- `SOAP` This directory contains the halo catalogues
- `HBTplus` This directory contains the full halo merger tree information for the simulation
- `output_list.txt` This file lists the redshifts at which snapshots have been created

Let's print the first few lines of `output_list.txt` 

In [15]:
!head -5 /cosma8/data/dp004/colibre/Runs/L0025N0376/Fiducial_test/output_list.txt

# Redshift, Select Output
30.000000, Snapshot
25.000000, Snipshot
22.500000, Snipshot
20.000000, Snapshot


We can see that the first snapshot (`snap_nr=0` since the files are 0-indexed) corresponds to `z=30`. If we want the `z=20` snapshot we will need to load `snap_nr=3`.

For some redshifts we output all the particle properties. For others we output a reduced file with significantly less properties, which are called snipshots.

# Snapshots

Analysis of SWIFT snapshots should be carried out using [swiftsimio](https://swiftsimio.readthedocs.io/en/latest/index.html). This ensures that all units will be handled correctly (internal cosmological simulation units can be strange), and contains a lot of useful functionality for working with the files.


Within the `snapshots` directory there is a directory for each snapshot. Each of these contains a number of "chunk" files, named `colibre_xxxx.y.hdf5`. There is also a virtual file called "colibre_xxxx.hdf5". **You should always load the virtual file rather than the chunk files**. Each chunk file only contains particles of a certain region of a simulation, but if you use the virtual file you will have access to all the particles. Even if you only want to load a small region of the simulation you should use the virtual file combined with swiftsimio's masking ability (see below).

In [12]:
simulation_dir = "/cosma8/data/dp004/colibre/Runs/L0025N0376/Fiducial_test"
# For this test run we only had 124 snapshots, but all the main simulations have 128 snapshots
snap_nr = 123


In [ ]:
Virtual files

output_list.txt

In [ ]:
output_fields.yml, data also available in swiftsimio

In [ ]:
- Visualising volume (for environment)

In [2]:
- Units

- Comoving vs physical

- Invalid ConversionError (workaround)

SyntaxError: invalid syntax (1399047486.py, line 1)

In [8]:
Masking (summary swift output structure)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4239921561.py, line 1)

In [ ]:
# Other simulation information
Redshifts

# SOAP

Link to current SOAP pdf

Tell us what properties you want

In [6]:
Datasets from pipeline, how to use

SyntaxError: invalid syntax (1570704140.py, line 1)

In [4]:
Reproduce pipeline plot

SyntaxError: invalid syntax (1626685914.py, line 1)

In [5]:
Subset writing?

Object `writing` not found.


# Merger trees

# SWIFT galaxy

In [2]:
Membership files

SyntaxError: invalid syntax (396452833.py, line 1)

In [ ]:
- Visualising objects